In [1]:
import pybaseball
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
def swing_type(outcome_text):
    out_text = outcome_text.lower()
    swing_pattern = [r'field_out', r'single', r'double', r'triple',
                     r'home_run', r'grounded_into_double_play', r'force_out', 
                     r'sac_fly', r'field_error', r'fielders_choice', r'fielders_choice_out',
                     r'double_play', r'triple_play', r'swinging_strike', r'foul', r'foul_tip', r'swinging_strike_blocked', r'blocked_ball'
    ]
    take_pattern = [r'ball', r'walk', r'hit_by_pitch', r'called_strike', r'intent_walk', r'called_strike']

    for p in swing_pattern:
        if p in out_text:
            return 1
    for p in take_pattern:
        if p in out_text:
            return 0
    return None 



In [3]:
batter_info = pybaseball.statcast('2025-10-14', '2025-10-15')
batter_info

This is a large query, it may take a moment to complete


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 50%|█████     | 1/2 [00:00<00:00,  1.66it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 2/2 [00:00<00:00,  3.17it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
167,FC,2025-10-15,90.3,3.66,5.66,"Fluharty, Mason",645302,689254,field_out,hit_into_play,...,<NA>,2.06,-0.4,0.4,<NA>,2.403379,30.452019,20.772955,40.169943,7.508847
173,FC,2025-10-15,91.6,3.67,5.67,"Fluharty, Mason",641598,689254,field_out,hit_into_play,...,<NA>,1.76,-0.09,0.09,<NA>,16.710821,6.718283,30.394547,43.844998,29.77832
181,FC,2025-10-15,90.0,3.54,5.65,"Fluharty, Mason",641598,689254,NaN,ball,...,<NA>,1.95,-0.32,0.32,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
188,ST,2025-10-15,81.1,3.53,5.77,"Fluharty, Mason",641598,689254,NaN,ball,...,<NA>,3.24,-1.2,1.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
197,FC,2025-10-15,91.7,3.73,5.69,"Fluharty, Mason",553993,689254,strikeout,swinging_strike,...,<NA>,1.54,-0.09,0.09,<NA>,-3.049613,27.407045,42.523556,33.052072,18.263231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,SL,2025-10-14,83.9,-3.66,5.24,"Peralta, Freddy",660271,642547,strikeout,swinging_strike,...,<NA>,3.05,-0.33,0.33,36.3,29.550436,-28.863768,45.627352,40.956874,42.490008
258,FF,2025-10-14,94.4,-3.69,5.18,"Peralta, Freddy",660271,642547,NaN,ball,...,<NA>,1.31,0.65,-0.65,33.1,<NA>,<NA>,<NA>,<NA>,<NA>
261,CU,2025-10-14,76.7,-3.7,5.54,"Peralta, Freddy",660271,642547,NaN,ball,...,<NA>,4.22,-0.57,0.57,35.8,<NA>,<NA>,<NA>,<NA>,<NA>
268,CU,2025-10-14,78.1,-3.59,5.56,"Peralta, Freddy",660271,642547,NaN,called_strike,...,<NA>,4.21,-0.48,0.48,38.7,<NA>,<NA>,<NA>,<NA>,<NA>


In [4]:
shortened_data = batter_info[['batter', 'pitch_type', 'description', 'plate_x', 'plate_z', 'events', 'release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'hc_x', 'hc_y']]
pruned_data = shortened_data
print(pruned_data.shape)
pruned_data

(558, 15)


,batter,pitch_type,description,plate_x,plate_z,events,release_speed,release_pos_x,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,hc_x,hc_y
167,645302,FC,hit_into_play,-0.03,3.5,field_out,90.3,3.66,80.7,6,91.4,2738,6.7,153.78,162.01
173,641598,FC,hit_into_play,0.5,2.66,field_out,91.6,3.67,94.1,68,92.7,2766,6.8,180.11,172.14
181,641598,FC,ball,-1.77,0.44,NaN,90.0,3.54,<NA>,<NA>,90.7,2876,6.9,<NA>,<NA>
188,641598,ST,ball,0.79,3.22,NaN,81.1,3.53,<NA>,<NA>,81.9,2737,7.0,<NA>,<NA>
197,553993,FC,swinging_strike,0.17,2.96,strikeout,91.7,3.73,<NA>,<NA>,92.8,2793,6.8,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,660271,SL,swinging_strike,-0.02,1.3,strikeout,83.9,-3.66,<NA>,<NA>,84.2,2242,6.6,<NA>,<NA>
258,660271,FF,ball,0.95,1.19,NaN,94.4,-3.69,<NA>,<NA>,94.1,2480,6.6,<NA>,<NA>
261,660271,CU,ball,-1.47,1.3,NaN,76.7,-3.7,<NA>,<NA>,76.6,2234,6.3,<NA>,<NA>
268,660271,CU,called_strike,-0.4,3.41,NaN,78.1,-3.59,<NA>,<NA>,77.9,2298,6.2,<NA>,<NA>


In [5]:
both_na = pruned_data['hc_x'].isna() & pruned_data['hc_y'].isna()
both_na_2 = pruned_data['launch_speed'].isna() & pruned_data['launch_angle'].isna()

pruned_data['outcome_text'] = pruned_data['events']

pruned_data['outcome_text'] = pruned_data['description'].where(both_na, pruned_data['events'])

pruned_data.loc[both_na_2, 'launch_speed'] = 0
pruned_data.loc[both_na_2, 'launch_angle'] = 0

pruned_data = pruned_data.dropna(subset=['outcome_text'])


C:\Users\yjain\AppData\Local\Temp\ipykernel_45728\1538957564.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['outcome_text'] = pruned_data['events']
C:\Users\yjain\AppData\Local\Temp\ipykernel_45728\1538957564.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['outcome_text'] = pruned_data['description'].where(both_na, pruned_data['events'])


In [7]:
pruned_data['batting_pattern'] = pruned_data['outcome_text'].apply(swing_type)
pruned_data = pruned_data.dropna(subset=['batting_pattern'])
pruned_data

,batter,pitch_type,description,plate_x,plate_z,events,release_speed,release_pos_x,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,hc_x,hc_y,outcome_text,batting_pattern
167,645302,FC,hit_into_play,-0.03,3.5,field_out,90.3,3.66,80.7,6,91.4,2738,6.7,153.78,162.01,field_out,1
173,641598,FC,hit_into_play,0.5,2.66,field_out,91.6,3.67,94.1,68,92.7,2766,6.8,180.11,172.14,field_out,1
181,641598,FC,ball,-1.77,0.44,NaN,90.0,3.54,0.0,0,90.7,2876,6.9,<NA>,<NA>,ball,0
188,641598,ST,ball,0.79,3.22,NaN,81.1,3.53,0.0,0,81.9,2737,7.0,<NA>,<NA>,ball,0
197,553993,FC,swinging_strike,0.17,2.96,strikeout,91.7,3.73,0.0,0,92.8,2793,6.8,<NA>,<NA>,swinging_strike,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,660271,SL,swinging_strike,-0.02,1.3,strikeout,83.9,-3.66,0.0,0,84.2,2242,6.6,<NA>,<NA>,swinging_strike,1
258,660271,FF,ball,0.95,1.19,NaN,94.4,-3.69,0.0,0,94.1,2480,6.6,<NA>,<NA>,ball,0
261,660271,CU,ball,-1.47,1.3,NaN,76.7,-3.7,0.0,0,76.6,2234,6.3,<NA>,<NA>,ball,0
268,660271,CU,called_strike,-0.4,3.41,NaN,78.1,-3.59,0.0,0,77.9,2298,6.2,<NA>,<NA>,called_strike,0


In [8]:
 # We may not need this as 'batter' is already a label, sike i lied
batter_enc = LabelEncoder() 
pruned_data['batter_id'] = batter_enc.fit_transform(pruned_data['batter'])
pitch_enc = LabelEncoder()
pruned_data['pitch_type_id'] = pitch_enc.fit_transform(pruned_data['pitch_type'])
outcome_enc = LabelEncoder()
pruned_data['outcome_id'] = outcome_enc.fit_transform(pruned_data['outcome_text'])
batter_pattern_enc = LabelEncoder()
pruned_data['batting_pattern_id'] = batter_pattern_enc.fit_transform(pruned_data['batting_pattern'])

NUM_BATTERS = pruned_data['batter'].nunique()
NUM_PITCHES = pruned_data['pitch_type_id'].nunique()
NUM_OUTCOMES = pruned_data['outcome_id'].nunique()
NUM_BATTER_PATTERNS = pruned_data['batting_pattern_id'].nunique()

scaler = StandardScaler()
pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']] = scaler.fit_transform(pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension']].astype(float))
pruned_data.head()

print(pruned_data['outcome_text'].value_counts())

#pruned_data = pruned_data[pruned_data['events'] != 'catcher_interf']

print(pruned_data['outcome_id'].value_counts())
print(pruned_data['outcome_text'].values)
labels = set(pruned_data['outcome_text'].values)
labels = list(labels)
print(labels)
print(NUM_OUTCOMES)
print(NUM_PITCHES)
print(NUM_BATTER_PATTERNS)

outcome_text
ball                         165
called_strike                 97
foul                          92
swinging_strike               69
field_out                     65
single                        20
blocked_ball                  14
home_run                      11
double                         9
swinging_strike_blocked        3
grounded_into_double_play      3
force_out                      3
foul_tip                       3
hit_by_pitch                   2
field_error                    1
Name: count, dtype: int64
outcome_id
0     165
2      97
7      92
13     69
5      65
12     20
1      14
11     11
3       9
14      3
9       3
6       3
8       3
10      2
4       1
Name: count, dtype: int64
['field_out' 'field_out' 'ball' 'ball' 'swinging_strike' 'foul' 'ball'
 'foul' 'ball' 'foul' 'foul' 'foul' 'called_strike' 'ball' 'single'
 'field_out' 'called_strike' 'single' 'field_out' 'field_out' 'home_run'
 'called_strike' 'ball' 'called_strike' 'ball' 'ball' 'foul'
 'call

In [10]:
class SuperDataSet(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __getitem__(self, index):
        row = self.df.loc[index]
        batter_id = int(row['batter_id'])
        pitch_type_id = int(row['pitch_type_id'])
        features = torch.tensor(row[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'batting_pattern_id']].values.astype(np.float32), dtype=torch.float32)
        y = int(row['outcome_id'])
        return batter_id, pitch_type_id, features, y
    def __len__(self):
        return len(self.df)




train_df, test_df = train_test_split(pruned_data, test_size=0.2, random_state=42)

train_dataset = SuperDataSet(train_df)
test_dataset = SuperDataSet(test_df)

print(f"Train dataset size: {len(train_dataset)}")
print(train_dataset[1])
print(f"Test dataset size: {len(test_dataset)}")
print(test_dataset[1])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)



Train dataset size: 445
(30, 2, tensor([-0.5738, -0.1241,  0.2362,  0.4379, -0.7091, -0.2246, -0.4631,  0.6489,
         0.2905,  1.0000]), 13)
Test dataset size: 112
(5, 4, tensor([ 0.4632, -0.3389,  1.0046, -1.3807, -0.7091, -0.2246,  0.4651, -2.3813,
         0.2905,  1.0000]), 13)


In [11]:
class SuperModel(nn.Module):
    def __init__(self, num_batters, num_pitch_types, num_input_data_types, output_dim, batter_embedding=10, pitch_embedding=5, hidden_dim=64): #Mess around with batter_embedding, pitch_embedding, hidden_dim
        super().__init__()
        self.batter_embedding = nn.Embedding(num_batters, batter_embedding)
        self.pitch_embedding = nn.Embedding(num_pitch_types, pitch_embedding)
        self.linear1 = nn.Linear(num_input_data_types, 32)
        self.linear2 = nn.Linear(32, 16)

        self.fc1 = nn.Linear(batter_embedding + pitch_embedding + 16, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, batter_id, pitch_id, input_data):
        b = self.batter_embedding(batter_id)
        p = self.pitch_embedding(pitch_id) 
        i = self.linear1(input_data)
        i = self.relu(i)
        i = self.linear2(i)
        i = self.relu(i)
        x = torch.cat([b, p, i], dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x # {'foul': 0.1, 'single': 0.3, ...}

In [12]:
model_t = SuperModel(num_batters=NUM_BATTERS, num_pitch_types=NUM_PITCHES, num_input_data_types=10, output_dim=NUM_OUTCOMES).to('cuda')
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)
print(model_t)


SuperModel(
  (batter_embedding): Embedding(41, 10)
  (pitch_embedding): Embedding(10, 5)
  (linear1): Linear(in_features=10, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=16, bias=True)
  (fc1): Linear(in_features=31, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=15, bias=True)
)


In [13]:
for epoch in range(20):
    model_t.train()
    for i, (batter_id, pitch_type_id, features, y) in enumerate(train_loader):
        batter_id = batter_id.to('cuda').long()
        pitch_type_id = pitch_type_id.to('cuda').long()
        features = features.to('cuda')
        y = y.to('cuda')
        logits = model_t(batter_id, pitch_type_id, features)
        loss_value = loss(logits, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss_value.item()}")



Epoch 1, Loss: 2.8632583618164062
Epoch 1, Loss: 2.8549771308898926
Epoch 1, Loss: 2.7030818462371826
Epoch 1, Loss: 2.6474602222442627
Epoch 1, Loss: 2.624507427215576
Epoch 1, Loss: 2.966217279434204
Epoch 1, Loss: 2.852351188659668
Epoch 1, Loss: 2.7200918197631836
Epoch 1, Loss: 2.793368101119995
Epoch 1, Loss: 2.643679141998291
Epoch 1, Loss: 2.5650765895843506
Epoch 1, Loss: 2.5307064056396484
Epoch 1, Loss: 2.6733274459838867
Epoch 1, Loss: 2.8721213340759277
Epoch 1, Loss: 2.5859055519104004
Epoch 1, Loss: 2.756441593170166
Epoch 1, Loss: 2.6747145652770996
Epoch 1, Loss: 2.4862570762634277
Epoch 1, Loss: 2.699305534362793
Epoch 1, Loss: 2.622572660446167
Epoch 1, Loss: 2.5035862922668457
Epoch 1, Loss: 2.5430312156677246
Epoch 1, Loss: 2.6809239387512207
Epoch 1, Loss: 2.2951560020446777
Epoch 1, Loss: 2.675636053085327
Epoch 1, Loss: 2.593116283416748
Epoch 1, Loss: 2.4498844146728516
Epoch 1, Loss: 2.821291446685791
Epoch 1, Loss: 2.786679267883301
Epoch 1, Loss: 2.707592487

In [19]:
model_t.eval()
with torch.inference_mode():
    all = []
    probable = []
    for i, (batter_id, pitch_type_id, features, y) in enumerate(test_loader):
        batter_id = batter_id.to('cuda').long()
        pitch_type_id = pitch_type_id.to('cuda').long()
        features = features.to('cuda')
        y = y.to('cuda')
        logits = model_t(batter_id, pitch_type_id, features)
        #print(logits)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        #print(probs)

        for batch_idx, probs_row in enumerate(probs):
            pred_dict = {}
            max_value = 0
            outcome_probable = None
            for i in range(len(labels)):
                if float(probs_row[i]) > max_value:
                    max_value = probs_row[i]
                    outcome_probable = labels[i]
                pred_dict[labels[i]] = float(probs_row[i])
            all.append(pred_dict)
            probable.append((max_value,outcome_probable))
            print(pred_dict)
        loss_value = loss(logits, y)
        #print(f"Test Batch {i+1}, Loss: {loss_value.item()}")

{'ball': 9.025148415275908e-07, 'grounded_into_double_play': 7.353347882599337e-06, 'blocked_ball': 0.0004355049168225378, 'force_out': 1.0203363848271252e-10, 'called_strike': 9.236437725768099e-10, 'single': 4.950561560690403e-06, 'field_error': 5.364326582935064e-08, 'double': 0.01393648236989975, 'home_run': 0.0005230847746133804, 'foul_tip': 3.3088703218359106e-12, 'field_out': 6.567199561935411e-11, 'swinging_strike_blocked': 5.98503677173845e-11, 'swinging_strike': 2.897603579299357e-08, 'foul': 0.9850558042526245, 'hit_by_pitch': 3.586421371437609e-05}
{'ball': 0.00010220839612884447, 'grounded_into_double_play': 0.003998114727437496, 'blocked_ball': 1.0233279681415297e-05, 'force_out': 1.1483914303712667e-14, 'called_strike': 3.0997293620771416e-10, 'single': 5.278449748402636e-07, 'field_error': 8.26463203429384e-11, 'double': 0.0003425943141337484, 'home_run': 0.00011708218517014757, 'foul_tip': 1.406019940232428e-13, 'field_out': 1.1465903920759857e-10, 'swinging_strike_blo

In [20]:
probable

[(np.float32(0.9850558), 'foul'),
 (np.float32(0.9954241), 'foul'),
 (np.float32(0.9808133), 'ball'),
 (np.float32(0.7013042), 'double'),
 (np.float32(0.86637425), 'foul'),
 (np.float32(0.6224339), 'swinging_strike'),
 (np.float32(0.8825047), 'foul'),
 (np.float32(0.9999678), 'ball'),
 (np.float32(0.7314753), 'single'),
 (np.float32(0.9747594), 'ball'),
 (np.float32(0.6993062), 'double'),
 (np.float32(0.9852785), 'ball'),
 (np.float32(0.8687862), 'ball'),
 (np.float32(0.7927304), 'ball'),
 (np.float32(0.584734), 'ball'),
 (np.float32(0.52744204), 'double'),
 (np.float32(0.5900309), 'double'),
 (np.float32(0.8773102), 'swinging_strike'),
 (np.float32(0.5551577), 'double'),
 (np.float32(0.8255082), 'ball'),
 (np.float32(0.916241), 'blocked_ball'),
 (np.float32(0.99710995), 'blocked_ball'),
 (np.float32(0.9798794), 'ball'),
 (np.float32(0.999213), 'ball'),
 (np.float32(0.9853419), 'blocked_ball'),
 (np.float32(0.7142248), 'swinging_strike'),
 (np.float32(0.578242), 'double'),
 (np.float32